<a href="https://colab.research.google.com/github/saerarawas/AAI_634O_A11_202520/blob/main/week3/Building_a_Basic_Data_Pipeline_With_Error_Handling_Rawas_Saera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pymongo
import pandas as pd

# Define the sales data
data = {
    "transaction_id": ["T001", "T002", "T003"],
    "customer_id": ["C001", "C002", "C001"],
    "product_id": ["P001", "P002", "P003"],
    "quantity": [2, 1, 3],
    "price": [100, 200, 50]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Save the DataFrame as a CSV file
csv_filename = "sales.csv"
df.to_csv(csv_filename, index=False)

print(f"CSV file '{csv_filename}' has been created successfully!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 16.8 MB/s eta 0:00:00
CSV file 'sales.csv' has been created successfully!


In [2]:
import pandas as pd
import logging
import pymongo
from pymongo import MongoClient

# Configure logging
logging.basicConfig(filename="pipeline.log", level=logging.INFO,
                    format="%(asctime)s - %(levelname)s - %(message)s")

# MongoDB Connection
MONGO_URI = "mongodb://localhost:27017/"
DB_NAME = "sales_db"
COLLECTION_NAME = "sales_data"

def extract_data(csv_file):
    """Extract data from a CSV file."""
    try:
        df = pd.read_csv(csv_file)
        logging.info(f"Successfully extracted data from {csv_file}")
        return df
    except FileNotFoundError:
        logging.error(f"File {csv_file} not found.")
        raise
    except Exception as e:
        logging.error(f"Error extracting data: {e}")
        raise

def transform_data(df):
    """Transform data: Compute total revenue (Quantity * Price)."""
    try:
        df["total_revenue"] = df["quantity"] * df["price"]
        logging.info("Successfully transformed data")
        return df
    except KeyError as e:
        logging.error(f"Missing column in data: {e}")
        raise
    except Exception as e:
        logging.error(f"Error transforming data: {e}")
        raise

def load_data(df):
    """Load data into MongoDB."""
    try:
        client = MongoClient(MONGO_URI)
        db = client[DB_NAME]
        collection = db[COLLECTION_NAME]

        records = df.to_dict(orient="records")
        collection.insert_many(records)

        logging.info(f"Successfully loaded {len(records)} records into MongoDB")
    except pymongo.errors.ConnectionError:
        logging.error("Could not connect to MongoDB.")
        raise
    except Exception as e:
        logging.error(f"Error loading data into MongoDB: {e}")
        raise

def etl_pipeline(csv_file):
    """ETL Pipeline execution."""
    try:
        df = extract_data(csv_file)
        df = transform_data(df)
        load_data(df)
        logging.info("ETL pipeline executed successfully")
    except Exception as e:
        logging.error(f"ETL pipeline failed: {e}")

# Run ETL pipeline
if __name__ == "__main__":
    etl_pipeline("sales.csv")

ERROR:root:ETL pipeline failed: module 'pymongo.errors' has no attribute 'ConnectionError'
